## Imports:

In [1]:
import scipy.io
import os
import pandas
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pylab as plt_lab
import matplotlib.gridspec as gridspec
import obspy
import pandas as pd

RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

In [2]:
import warnings
warnings.filterwarnings('ignore')

## Load data:
We are going to use this data to plot spectograms of SCG signals. 

In [3]:
dir_path = "/Users/ecem/Documents/GitHub/seismocardiogram/data/diseased-dataset/Raw_Recordings"

In [4]:
mr_ = np.load(dir_path + "/trimmed_t_mr.npy", allow_pickle= True)
ms_ = np.load(dir_path + "/trimmed_t_ms.npy", allow_pickle= True)
as_ = np.load(dir_path + "/trimmed_t_as.npy", allow_pickle= True)
ar_ = np.load(dir_path + "/trimmed_t_ar.npy", allow_pickle= True)

In [5]:
print(ms_.shape)
print(mr_.shape)
print(ar_.shape)
print(as_.shape)

(3, 143, 2560)
(3, 591, 2560)
(3, 242, 2560)
(3, 484, 2560)


## Zero Crossing Rate:
Compute the zero-crossing rate of an audio time series.
The zero-crossing rate (ZCR) is the rate at which a signal changes from positive to zero to negative or from negative to zero to positive.

In [6]:
import librosa as lbs

In [7]:
def zero_crossing(array):
    zero_crossing_rate = np.ones(shape = (array.shape[0],array.shape[1], 7))
    
    for i in range(array.shape[1]):
        zero_crossing_rate[0][i][0:6] = lbs.feature.zero_crossing_rate(np.array(array[0][i]))
        zero_crossing_rate[1][i][0:6] = lbs.feature.zero_crossing_rate(np.array(array[1][i]))
        zero_crossing_rate[2][i][0:6] = lbs.feature.zero_crossing_rate(np.array(array[2][i]))

        zero_crossing_rate[0][i][6] = sum(lbs.zero_crossings(np.array(array[0][i])))
        zero_crossing_rate[1][i][6] = sum(lbs.zero_crossings(np.array(array[1][i])))
        zero_crossing_rate[2][i][6] = sum(lbs.zero_crossings(np.array(array[2][i])))
        
    return zero_crossing_rate

        

In [8]:
zcr_mr = zero_crossing(mr_)
zcr_ms = zero_crossing(ms_)
zcr_as = zero_crossing(as_)
zcr_ar = zero_crossing(ar_)

In [9]:
print(zcr_mr.shape)
print(zcr_ms.shape)
print(zcr_as.shape)
print(zcr_ar.shape)

(3, 591, 7)
(3, 143, 7)
(3, 484, 7)
(3, 242, 7)


In [10]:
zcr_mr[0][6][4]

0.10498046875

In [11]:
columns_zcr = []
for i in range(zcr_ms.shape[2]-1):
    columns_zcr.append("zcr_x{}".format(i))
    columns_zcr.append("zcr_y{}".format(i))
    columns_zcr.append("zcr_z{}".format(i))
    
columns_zcr.append("#ofcrossing_x")
columns_zcr.append("#ofcrossing_y")
columns_zcr.append("#ofcrossing_z")

print(columns_zcr)

['zcr_x0', 'zcr_y0', 'zcr_z0', 'zcr_x1', 'zcr_y1', 'zcr_z1', 'zcr_x2', 'zcr_y2', 'zcr_z2', 'zcr_x3', 'zcr_y3', 'zcr_z3', 'zcr_x4', 'zcr_y4', 'zcr_z4', 'zcr_x5', 'zcr_y5', 'zcr_z5', '#ofcrossing_x', '#ofcrossing_y', '#ofcrossing_z']


In [12]:
def dataframe_creator(array, disease = "None", df = pd.DataFrame(), l = 7, columns = None):
    values = []
    dis = []
    dis.append(disease)
    for i in range(array.shape[1]):
        llist = list(array[0][i][0:l-1].tolist() + array[1][i][0:l-1].tolist() 
                        + array [2][i][0:l-1].tolist())
        llist.append(array[0][i][l-1])
        llist.append(array[1][i][l-1])
        llist.append(array[2][i][l-1])
        llist.append(disease)

        values.append(llist)
        medium = pd.DataFrame([values[i]], columns = columns + ["disease"])
        
        
        df = pd.concat([df, medium], axis = 0)
    return df 

In [13]:
df = dataframe_creator(zcr_mr, disease = "MR", 
                         df =pd.DataFrame(columns = columns_zcr), columns = columns_zcr)
df = dataframe_creator(zcr_ar, disease = "AR", df = df, columns = columns_zcr )
df = dataframe_creator(zcr_ms, disease = "MS", df = df, columns = columns_zcr )
df = dataframe_creator(zcr_as, disease = "AS", df = df, columns = columns_zcr )

df.reset_index(drop = True, inplace = True)
df

,zcr_x0,zcr_y0,zcr_z0,zcr_x1,zcr_y1,zcr_z1,zcr_x2,zcr_y2,zcr_z2,zcr_x3,...,zcr_x4,zcr_y4,zcr_z4,zcr_x5,zcr_y5,zcr_z5,#ofcrossing_x,#ofcrossing_y,#ofcrossing_z,disease
0,0.082031,0.129395,0.178711,0.195312,0.149414,0.102051,0.047363,0.083984,0.11084,0.118652,...,0.082031,0.14209,0.190918,0.206543,0.162598,0.103027,475.0,289.0,503.0,MR
1,0.095215,0.140625,0.179199,0.173828,0.128418,0.083008,0.064453,0.096191,0.125,0.130859,...,0.104004,0.159668,0.220215,0.236328,0.179199,0.123535,459.0,333.0,581.0,MR
2,0.091309,0.135254,0.18457,0.183594,0.137695,0.094238,0.05957,0.095215,0.12793,0.128906,...,0.126953,0.181152,0.235352,0.231445,0.162598,0.108398,471.0,333.0,595.0,MR
3,0.08252,0.126953,0.171875,0.177246,0.135742,0.091797,0.066895,0.100098,0.128418,0.120605,...,0.120117,0.169434,0.219238,0.20752,0.151367,0.101562,449.0,307.0,557.0,MR
4,0.076172,0.117188,0.160645,0.173828,0.131836,0.09082,0.065918,0.100586,0.125488,0.118652,...,0.11084,0.155273,0.200195,0.192383,0.13623,0.091309,427.0,315.0,507.0,MR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,0.094727,0.138672,0.180176,0.179199,0.134766,0.09082,0.058105,0.080078,0.09375,0.085938,...,0.120605,0.173828,0.235352,0.22998,0.175293,0.12207,471.0,239.0,607.0,AS
1456,0.092773,0.126953,0.160645,0.164551,0.114746,0.081055,0.050293,0.07666,0.099609,0.101562,...,0.113281,0.167969,0.222168,0.220703,0.165527,0.111328,427.0,259.0,573.0,AS
1457,0.069824,0.111328,0.155273,0.147949,0.11084,0.069336,0.042969,0.067871,0.093262,0.100586,...,0.101074,0.162598,0.219727,0.220215,0.170898,0.109863,371.0,241.0,559.0,AS
1458,0.071777,0.092285,0.123535,0.137695,0.097656,0.077637,0.0625,0.093262,0.129395,0.125,...,0.110352,0.160645,0.22168,0.224609,0.167969,0.117676,349.0,315.0,571.0,AS


### Chromogram:
We can use Chroma feature visualization to know how dominant the characteristics of a certain pitch {C, C♯, D, D♯, E, F, F♯, G, G♯, A, A♯, B} is present in the sampled frame.

Compute a chromagram from a waveform or power spectrogram.
It returns Normalized energy for each chroma bin at each frame.

Hop size should refer to the number of samples in between successive frames. For signal analysis Hop Size should be less than the frame size, so that frames overlap.

In [14]:
import warnings
warnings.filterwarnings('ignore')

In [15]:
def chromagram_average(array):
    pitch = np.ones(shape = (array.shape[0],array.shape[1], 12)) 
    
    for i in range(array.shape[1]):
        for j in range(12):
            pitch[0][i][j] = lbs.feature.chroma_stft(np.array(array[0][i]), sr= 256, hop_length = 512)[j].mean()
            pitch[1][i][j] = lbs.feature.chroma_stft(np.array(array[1][i]), sr= 256, hop_length = 512)[j].mean()
            pitch[2][i][j] = lbs.feature.chroma_stft(np.array(array[2][i]), sr= 256, hop_length = 512)[j].mean()

    return pitch

In [16]:
chro_mr = chromagram_average(mr_)
chro_ms = chromagram_average(ms_)
chro_as = chromagram_average(as_)
chro_ar = chromagram_average(ar_)

In [17]:
print(chro_mr.shape)

(3, 591, 12)


In [18]:
lst = ["C", "C♯", "D", "D♯", "E", "F", "F♯", "G", "G♯", "A", "A♯", "B"]

In [19]:
col_chro = []
for i in range(chro_mr.shape[2]):
    col_chro.append(lst[i]+ "_x")
    col_chro.append(lst[i]+ "_y")
    col_chro.append(lst[i]+ "_z")
    
print(col_chro)
print(len(col_chro))

['C_x', 'C_y', 'C_z', 'C♯_x', 'C♯_y', 'C♯_z', 'D_x', 'D_y', 'D_z', 'D♯_x', 'D♯_y', 'D♯_z', 'E_x', 'E_y', 'E_z', 'F_x', 'F_y', 'F_z', 'F♯_x', 'F♯_y', 'F♯_z', 'G_x', 'G_y', 'G_z', 'G♯_x', 'G♯_y', 'G♯_z', 'A_x', 'A_y', 'A_z', 'A♯_x', 'A♯_y', 'A♯_z', 'B_x', 'B_y', 'B_z']
36


In [20]:
ch_df = dataframe_creator(chro_mr, disease = "MR", 
                         df =pd.DataFrame(columns = col_chro), columns = col_chro, l = 12)
ch_df = dataframe_creator(chro_ar, disease = "AR", df = ch_df, columns = col_chro, l = 12)
ch_df = dataframe_creator(chro_ms, disease = "MS", df = ch_df, columns = col_chro, l = 12)
ch_df = dataframe_creator(chro_as, disease = "AS", df = ch_df, columns = col_chro, l = 12)

ch_df.reset_index(drop = True, inplace = True)
ch_df

,C_x,C_y,C_z,C♯_x,C♯_y,C♯_z,D_x,D_y,D_z,D♯_x,...,A_x,A_y,A_z,A♯_x,A♯_y,A♯_z,B_x,B_y,B_z,disease
0,0.785613,0.724123,0.744865,0.841634,0.728695,0.666317,0.629508,0.69185,0.744346,0.811383,...,0.550876,0.488099,0.473361,0.52654,0.718695,0.840319,0.977623,1.0,1.0,MR
1,0.813454,0.690039,0.6054,0.670928,0.589845,0.58442,0.740568,0.789188,0.625764,0.720067,...,0.616335,0.558207,0.525845,0.585564,0.776684,0.942876,1.0,0.981521,0.967486,MR
2,0.768598,0.728458,0.828923,0.873236,0.671514,0.651726,0.56553,0.623666,0.701558,0.785699,...,0.528083,0.472831,0.417259,0.460455,0.660383,0.915534,0.982367,0.969857,1.0,MR
3,0.764322,0.678977,0.676762,0.733744,0.682311,0.612956,0.585404,0.627897,0.687111,0.794035,...,0.514232,0.449276,0.471934,0.545451,0.709124,0.834021,0.982914,0.909575,1.0,MR
4,0.814171,0.718093,0.686437,0.728895,0.682535,0.577518,0.665602,0.637256,0.653776,0.811659,...,0.412994,0.469447,0.453689,0.523012,0.759921,0.824143,0.96462,0.875552,0.993204,MR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,0.695537,0.577904,0.581972,0.603314,0.756066,0.701273,0.7496,0.81405,0.868236,0.923661,...,0.70158,0.647582,0.687628,0.740964,0.928917,0.95299,0.919111,1.0,0.93353,AS
1456,0.633613,0.540631,0.575075,0.558003,0.616585,0.66734,0.660269,0.709414,0.811116,0.771655,...,0.697578,0.694199,0.780676,0.807119,0.854429,0.922914,0.962602,0.932648,0.914875,AS
1457,0.647011,0.552037,0.550079,0.639667,0.658231,0.627622,0.671694,0.731769,0.762082,0.805286,...,0.676887,0.637471,0.764013,0.76671,0.899584,1.0,0.998977,0.993931,0.902801,AS
1458,0.66444,0.533446,0.551154,0.593311,0.606013,0.619207,0.665732,0.791669,0.878348,0.951537,...,0.827662,0.807564,0.778463,0.792535,0.777422,0.859151,0.910143,1.0,0.987707,AS


## Tempogram

Compute the tempogram: local autocorrelation of the onset strength envelope.

# kendime not tempogram değerlerini spektogram değerleri gibi kaydet.

In [21]:
import librosa.display

In [22]:
def tempo_average(array): # 384, 11
    means= []
    for i in range(array.shape[0]): #384
        means.append(array[i][0:11].mean())
    return np.array(means) # 384
        

In [23]:
def tempogram(array = 0, hop_lenght = 256):
    hop_length = 256
    sr =256
    oenv = librosa.onset.onset_strength(y=np.array(array), 
                                            sr=sr, 
                                            hop_length=hop_length)

    times = librosa.times_like(oenv, sr=sr, hop_length=hop_length)

    tempogram = librosa.feature.tempogram(onset_envelope=oenv, sr= sr,
                                              hop_length=hop_length)
    # Estimate the global tempo for display purposes
    tempo = librosa.beat.tempo(onset_envelope=oenv, sr= sr,
                                   hop_length=hop_length)[0]
    return [tempogram, tempo]

In [24]:
tempogram(ms_[0][0])[0].shape

(384, 11)

In [25]:
tempogram(ms_[0][0])[0][0]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [26]:
tempo_average(tempogram(ms_[0][0])[0]).shape

(384,)

In [27]:
tempo_average(tempogram(ms_[0][0])[0])[1]

0.6930136041001767

In [28]:
tempogram(ms_[0][0])[0][1].mean()

0.6930136041001767

In [29]:
def temp_values(array):
    final = np.ones(shape = (3, array.shape[1], 384))
    for i in range(array.shape[1]): #143
        
        _x = tempogram(array[0][i])[0] #1,2560
        _y = tempogram(array[1][i])[0]
        _z = tempogram(array[2][i])[0]

        final[0][i] = tempo_average(_x) #384
        final[1][i] = tempo_average(_y)
        final[2][i] = tempo_average(_z)
        
        
    return final

In [30]:
ms = temp_values(ms_)
mr = temp_values(mr_)
ar = temp_values(ar_)
ass = temp_values(as_)

In [31]:
ms.shape

(3, 143, 384)

In [32]:
print(ms.shape)
print(mr.shape)
print(ar.shape)
print(ass.shape)

(3, 143, 384)
(3, 591, 384)
(3, 242, 384)
(3, 484, 384)


In [33]:
columns_t = []
for i in range(ms.shape[-1]):
    columns_t.append("tempogram_x{}".format(i))
    columns_t.append("tempogram_y{}".format(i))
    columns_t.append("tempogram_z{}".format(i))
print(columns_t)

['tempogram_x0', 'tempogram_y0', 'tempogram_z0', 'tempogram_x1', 'tempogram_y1', 'tempogram_z1', 'tempogram_x2', 'tempogram_y2', 'tempogram_z2', 'tempogram_x3', 'tempogram_y3', 'tempogram_z3', 'tempogram_x4', 'tempogram_y4', 'tempogram_z4', 'tempogram_x5', 'tempogram_y5', 'tempogram_z5', 'tempogram_x6', 'tempogram_y6', 'tempogram_z6', 'tempogram_x7', 'tempogram_y7', 'tempogram_z7', 'tempogram_x8', 'tempogram_y8', 'tempogram_z8', 'tempogram_x9', 'tempogram_y9', 'tempogram_z9', 'tempogram_x10', 'tempogram_y10', 'tempogram_z10', 'tempogram_x11', 'tempogram_y11', 'tempogram_z11', 'tempogram_x12', 'tempogram_y12', 'tempogram_z12', 'tempogram_x13', 'tempogram_y13', 'tempogram_z13', 'tempogram_x14', 'tempogram_y14', 'tempogram_z14', 'tempogram_x15', 'tempogram_y15', 'tempogram_z15', 'tempogram_x16', 'tempogram_y16', 'tempogram_z16', 'tempogram_x17', 'tempogram_y17', 'tempogram_z17', 'tempogram_x18', 'tempogram_y18', 'tempogram_z18', 'tempogram_x19', 'tempogram_y19', 'tempogram_z19', 'tempogra

In [34]:
len(columns_t)

1152

In [35]:
mr[0][14][190]

2.5249426648626094e-06

In [36]:
df_t = dataframe_creator(mr, disease = "MR", 
                         df =pd.DataFrame(columns = columns_t), columns = columns_t, l = 384)
df_t = dataframe_creator(ar, disease = "AR", df = df_t, columns = columns_t, l = 384)
df_t = dataframe_creator(ms, disease = "MS", df = df_t, columns = columns_t, l = 384)
df_t = dataframe_creator(ass, disease = "AS", df = df_t, columns = columns_t, l = 384)

df_t.reset_index(drop = True, inplace = True)
df_t

,tempogram_x0,tempogram_y0,tempogram_z0,tempogram_x1,tempogram_y1,tempogram_z1,tempogram_x2,tempogram_y2,tempogram_z2,tempogram_x3,...,tempogram_x381,tempogram_y381,tempogram_z381,tempogram_x382,tempogram_y382,tempogram_z382,tempogram_x383,tempogram_y383,tempogram_z383,disease
0,1.0,0.630862,0.460418,0.398011,0.373869,0.370182,0.366289,0.362378,0.358451,0.354509,...,0.0,0.0,-0.0,0.0,0.0,0.0,0.0,0.0,0.0,MR
1,1.0,0.633055,0.469917,0.433266,0.42265,0.409615,0.40525,0.400866,0.396466,0.392049,...,0.0,0.0,-0.0,0.0,0.0,0.0,0.0,0.0,0.0,MR
2,1.0,0.667625,0.523938,0.495735,0.508781,0.500779,0.495302,0.489806,0.484291,0.478759,...,0.0,0.0,-0.0,0.0,-0.0,-0.0,0.0,0.0,0.0,MR
3,1.0,0.580587,0.402545,0.356726,0.341081,0.323889,0.32052,0.317136,0.313737,0.310323,...,0.0,0.0,-0.0,0.0,0.0,0.0,0.0,0.0,0.0,MR
4,1.0,0.663031,0.520847,0.471459,0.454914,0.454337,0.449446,0.444535,0.439606,0.43466,...,0.0,0.0,-0.0,0.0,-0.0,-0.0,0.0,0.0,0.0,MR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1.0,0.734361,0.632793,0.60423,0.594719,0.587424,0.580928,0.574409,0.567869,0.561311,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,AS
1456,1.0,0.641136,0.493076,0.449146,0.436958,0.430718,0.426101,0.421466,0.416814,0.412145,...,0.0,0.0,-0.0,0.0,0.0,0.0,0.0,0.0,0.0,AS
1457,1.0,0.716621,0.596688,0.566692,0.569601,0.573941,0.567578,0.561192,0.554787,0.548365,...,0.0,0.0,-0.0,0.0,-0.0,0.0,0.0,0.0,0.0,AS
1458,1.0,0.651434,0.479616,0.448806,0.463003,0.470679,0.465546,0.460394,0.455224,0.450038,...,0.0,0.0,-0.0,0.0,0.0,0.0,0.0,0.0,0.0,AS


In [37]:
df_t.isin([0]).sum(axis=0)

tempogram_x0      0
tempogram_y0      0
tempogram_z0      0
tempogram_x1      0
tempogram_y1      0
                 ..
tempogram_z382    0
tempogram_x383    0
tempogram_y383    0
tempogram_z383    0
disease           0
Length: 1153, dtype: int64

In [38]:
df_t['tempogram_x382'][10]

4.465718366293722e-17

In [39]:
final_df = pd.concat([df.drop(['disease'], axis =1), ch_df.drop(['disease'], axis =1)], axis=1)
final_df

,zcr_x0,zcr_y0,zcr_z0,zcr_x1,zcr_y1,zcr_z1,zcr_x2,zcr_y2,zcr_z2,zcr_x3,...,G♯_z,A_x,A_y,A_z,A♯_x,A♯_y,A♯_z,B_x,B_y,B_z
0,0.082031,0.129395,0.178711,0.195312,0.149414,0.102051,0.047363,0.083984,0.11084,0.118652,...,0.57539,0.550876,0.488099,0.473361,0.52654,0.718695,0.840319,0.977623,1.0,1.0
1,0.095215,0.140625,0.179199,0.173828,0.128418,0.083008,0.064453,0.096191,0.125,0.130859,...,0.755567,0.616335,0.558207,0.525845,0.585564,0.776684,0.942876,1.0,0.981521,0.967486
2,0.091309,0.135254,0.18457,0.183594,0.137695,0.094238,0.05957,0.095215,0.12793,0.128906,...,0.510877,0.528083,0.472831,0.417259,0.460455,0.660383,0.915534,0.982367,0.969857,1.0
3,0.08252,0.126953,0.171875,0.177246,0.135742,0.091797,0.066895,0.100098,0.128418,0.120605,...,0.533524,0.514232,0.449276,0.471934,0.545451,0.709124,0.834021,0.982914,0.909575,1.0
4,0.076172,0.117188,0.160645,0.173828,0.131836,0.09082,0.065918,0.100586,0.125488,0.118652,...,0.522988,0.412994,0.469447,0.453689,0.523012,0.759921,0.824143,0.96462,0.875552,0.993204
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,0.094727,0.138672,0.180176,0.179199,0.134766,0.09082,0.058105,0.080078,0.09375,0.085938,...,0.741646,0.70158,0.647582,0.687628,0.740964,0.928917,0.95299,0.919111,1.0,0.93353
1456,0.092773,0.126953,0.160645,0.164551,0.114746,0.081055,0.050293,0.07666,0.099609,0.101562,...,0.750513,0.697578,0.694199,0.780676,0.807119,0.854429,0.922914,0.962602,0.932648,0.914875
1457,0.069824,0.111328,0.155273,0.147949,0.11084,0.069336,0.042969,0.067871,0.093262,0.100586,...,0.671522,0.676887,0.637471,0.764013,0.76671,0.899584,1.0,0.998977,0.993931,0.902801
1458,0.071777,0.092285,0.123535,0.137695,0.097656,0.077637,0.0625,0.093262,0.129395,0.125,...,0.766803,0.827662,0.807564,0.778463,0.792535,0.777422,0.859151,0.910143,1.0,0.987707


In [40]:
final_df = pd.concat([final_df, df_t], axis=1)
final_df

,zcr_x0,zcr_y0,zcr_z0,zcr_x1,zcr_y1,zcr_z1,zcr_x2,zcr_y2,zcr_z2,zcr_x3,...,tempogram_x381,tempogram_y381,tempogram_z381,tempogram_x382,tempogram_y382,tempogram_z382,tempogram_x383,tempogram_y383,tempogram_z383,disease
0,0.082031,0.129395,0.178711,0.195312,0.149414,0.102051,0.047363,0.083984,0.11084,0.118652,...,0.0,0.0,-0.0,0.0,0.0,0.0,0.0,0.0,0.0,MR
1,0.095215,0.140625,0.179199,0.173828,0.128418,0.083008,0.064453,0.096191,0.125,0.130859,...,0.0,0.0,-0.0,0.0,0.0,0.0,0.0,0.0,0.0,MR
2,0.091309,0.135254,0.18457,0.183594,0.137695,0.094238,0.05957,0.095215,0.12793,0.128906,...,0.0,0.0,-0.0,0.0,-0.0,-0.0,0.0,0.0,0.0,MR
3,0.08252,0.126953,0.171875,0.177246,0.135742,0.091797,0.066895,0.100098,0.128418,0.120605,...,0.0,0.0,-0.0,0.0,0.0,0.0,0.0,0.0,0.0,MR
4,0.076172,0.117188,0.160645,0.173828,0.131836,0.09082,0.065918,0.100586,0.125488,0.118652,...,0.0,0.0,-0.0,0.0,-0.0,-0.0,0.0,0.0,0.0,MR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,0.094727,0.138672,0.180176,0.179199,0.134766,0.09082,0.058105,0.080078,0.09375,0.085938,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,AS
1456,0.092773,0.126953,0.160645,0.164551,0.114746,0.081055,0.050293,0.07666,0.099609,0.101562,...,0.0,0.0,-0.0,0.0,0.0,0.0,0.0,0.0,0.0,AS
1457,0.069824,0.111328,0.155273,0.147949,0.11084,0.069336,0.042969,0.067871,0.093262,0.100586,...,0.0,0.0,-0.0,0.0,-0.0,0.0,0.0,0.0,0.0,AS
1458,0.071777,0.092285,0.123535,0.137695,0.097656,0.077637,0.0625,0.093262,0.129395,0.125,...,0.0,0.0,-0.0,0.0,0.0,0.0,0.0,0.0,0.0,AS


In [41]:
csv_path = "/Users/ecem/Desktop/csv of scgs"

In [42]:
final_df.to_csv(csv_path + '/signal_processing.csv')